## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-10-51-36 +0000,bbc,Reeves decides against raising income tax rates,https://www.bbc.com/news/articles/c0mx3nv7zy8o...
1,2025-11-14-10-51-22 +0000,bbc,Why is Donald Trump threatening to sue the BBC?,https://www.bbc.com/news/articles/c0mx28vlp4wo...
2,2025-11-14-10-49-00 +0000,wsj,EU Exports to U.S. Jump After Trade Deal,https://www.wsj.com/economy/trade/eu-exports-t...
3,2025-11-14-10-36-59 +0000,nyt,Zelensky’s Image Is Stained as Corruption Inqu...,https://www.nytimes.com/2025/11/14/world/europ...
4,2025-11-14-10-34-53 +0000,wapo,China’s new aircraft supercarrier challenges U...,https://www.washingtonpost.com/world/2025/11/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,35
26,new,20
571,california,15
106,epstein,14
59,ai,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
274,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,99
254,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,81
300,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...,78
284,2025-11-13-15-27-26 +0000,bbc,What we know about new Epstein emails that men...,https://www.bbc.com/news/articles/c3rj0d97ynvo...,75
90,2025-11-14-03-04-50 +0000,nypost,Va. prosecutor accused of ‘stymying the invest...,https://nypost.com/2025/11/13/us-news/va-prose...,70


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
274,99,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
182,57,2025-11-13-21-46-58 +0000,nypost,Stocks have worst day in a month amid AI conce...,https://nypost.com/2025/11/13/business/stocks-...
76,51,2025-11-14-04-38-11 +0000,nypost,JD Vance says surge of illegal immigrants drov...,https://nypost.com/2025/11/13/us-news/jd-vance...
254,46,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
300,43,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...
86,41,2025-11-14-03-37-55 +0000,nypost,NYC Mayor-elect Zohran Mamdani holds secretive...,https://nypost.com/2025/11/13/us-news/nyc-mayo...
82,36,2025-11-14-03-56-00 +0000,nypost,Devastating blaze rips through historic Long I...,https://nypost.com/2025/11/13/us-news/devastat...
50,34,2025-11-14-08-35-34 +0000,nypost,Food influencer Michael Duarte’s wife breaks s...,https://nypost.com/2025/11/14/us-news/michael-...
112,34,2025-11-14-01-18-01 +0000,nypost,Canada man hijacks city bus but miraculously m...,https://nypost.com/2025/11/13/world-news/canad...
261,34,2025-11-13-17-01-23 +0000,nypost,Rep. Anna Paulina Luna launches probe into wil...,https://nypost.com/2025/11/13/us-news/rep-anna...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
